## Ejercicios Web Scraping BeautifulSoup

In [13]:
import numpy as np
import pandas as pd

import requests
import bs4 # Para ver la versión
from bs4 import BeautifulSoup

from time import sleep

In [ ]:
# Versiones

print(f"numpy=={np.__version__}")
print(f"pandas=={pd.__version__}")
print(f"requests=={requests.__version__}")
print(f"bs4=={bs4.__version__}")

De la pagina:

https://www.recetasgratis.net/Recetas-de-Ensaladas-listado_receta-4_1.html

- Sacar la siguiente información de las recetas de la primera página:

1. **Nombre**
2. **Descripcion**
3. **Comensales**
4. **Tiempo**
5. **Categoria**
6. **Dificultad**
7. **Ingredientes**
8. **Instrucciones**

Guardalo en un DataFrame y luego en un **`.csv`**.

<body class="ctrl-search action-index">
<div class="container" id="top">
<header class="header-layout">
<nav class="menu" data-js-selector="menu">
<div class="cerrar-menu"></div>
<div class="wrap-menu max_width clear">
<div class="descubre">
<div class="table">
<div class="centrar">Descubre
<svg fill="#ffffff" transform="rotate(90)" viewbox="0 0 330 330">
<path d="M250.606,154.389l-150-149.996c-5.857-5.858-15.355-5.858-21.213,0.001 c-5.857,5.858-5.857,15.355,0.001,21.213l139.393,139.39L79.393,304.394c-5.857,5.858-5.857,15.355,0.001,21.213 C82.322,328.536,86.161,330,90,330s7.678-1.464,10.607-4.394l149.999-150.004c2.814-2.813,4.394-6.628,4.394-10.606 C255,161.018,253.42,157.202,250.606,154.389z"></path>
</svg>
</div>
</div>
</div>
<div class="lista_menu">
<div class="clear">
<a class="ga" data-category="Header" data-event="Go to hub" data-label="1" href="https://www.recetasgratis.net/Recetas-de-Aperitivos-tapas-listado_receta-1_1.html"><div class="hubicon hubicon--menu hubicon--aperitivo

In [24]:
primera_receta_url = soup.find("a", class_="titulo titulo--resultado")["href"]
primera_receta_url

'https://www.recetasgratis.net/receta-de-ensalada-rusa-chilena-77853.html'

In [31]:
response = requests.get(primera_receta_url)
soup = BeautifulSoup(response.text, "html.parser")
soup.body

In [ ]:
#titulo
titulo = soup.find("h1", class_="titulo titulo--articulo").text

In [ ]:
#descripcion
#opcion 1 como es el primer parrafo se puede buscar soup.find("p")
descripcion = soup.find("div", class_="intro").find("p").text

In [ ]:
#comensales
comensales = soup.find("span", class_="property comensales").text
comensales_num = int(comensales.split(" ")[0])

In [32]:
#tiempo
tiempo = soup.find("span", class_="property duracion").text
tiempo_minutos = obtener_minutos(tiempo)

In [ ]:
#categoria
categoria = soup.find("span", class_="property para").text

In [ ]:
#dificultad
dificultad = soup.find("span", class_="property dificultad").text.lower().replace("dificultad ", "")

In [ ]:
#ingredientes
ingredientes = [ingrediente.text.strip() for ingrediente in soup.find("div", class_="ingredientes").find_all("label")]


In [ ]:
#instrucciones
# pasos = [paso.text for paso in soup.find_all("div", class_= "apartado")]
pasos = [paso.text for paso in soup.select(".apartado .orden + p")]

In [40]:
num_paginas = range(1, 3) #paginas 1 y 2
recetas_urls = []
for num_pagina in num_paginas:     
    url = f"https://www.recetasgratis.net/Recetas-de-Ensaladas-listado_receta-4_{num_pagina}.html"
    response = requests.get(url) 
    soup = BeautifulSoup(response.text, "html.parser")
    recetas_urls.extend([receta["href"] for receta in soup.find_all("a", class_= "titulo titulo--resultado")])

print(f"len recetas_urls {len(recetas_urls)}")

recetas_ok_urls = [url for url in recetas_urls if "receta-" in url]
print(f"len recetas_ok_urls {len(recetas_ok_urls)}")

len recetas_urls 100
len recetas_ok_urls 97


In [55]:
titulos, descripciones, comensales, tiempos = list(), list(), list(), list()
categorias, dificultades, ingredientes, pasos = list(), list(), list(), list()

In [56]:

for url in  recetas_ok_urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser") 
    titulo = soup.find("h1", class_="titulo titulo--articulo").text
    titulos.append(titulo)
    descripcion = soup.find("div", class_="intro").find("p").text
    descripciones.append(descripcion)
    comensal = soup.find("span", class_="property comensales").text
    comensales_num = int(comensal.split(" ")[0])
    comensales.append(comensales_num)
    tiempos.append(obtener_minutos(soup.find("span", class_="property duracion").text))
    try:
        categorias.append(soup.find("span", class_="property para").text)
    except:
        categorias.append(np.nan)
    dificultad = soup.find("span", class_="property dificultad").text.lower().replace("dificultad ", "")
    dificultades.append(dificultad)
    ingredientes1 = [ingrediente.text.strip() for ingrediente in soup.find("div", class_="ingredientes").find_all("label")]
    ingredientes.append(ingredientes1)
    pasos1 = [paso.text for paso in soup.select(".apartado .orden + p")]
    pasos.append(pasos1)
    

df = pd.DataFrame()    
df["titulos"] = titulos
df["descripcion"] = descripciones
df["comensales"] = comensales
df["tiempo_minutos"] = tiempos
df["categoria"] = categorias
df["dificultad"] = dificultades
df["ingredientes"] = ingredientes
df["pasos"] = pasos
df.head()

,titulos,descripcion,comensales,tiempo_minutos,categoria,dificultad,ingredientes,pasos
0,Receta de Ensalada rusa chilena,La ensalada rusa chilena es un acompañamiento ...,5,30,Acompañamiento,muy baja,"[5 papas, 2 zanahorias, ½ taza de arvejas coci...","[Lava y pela las zanahorias y papas., Corta la..."
1,Receta de Soltero de habas,"El soltero de habas es un plato peruano, espec...",5,30,NaN,baja,"[½ cebolla, 1 tomate, 200 gramos de queso fres...",[Comienza picando todas las verduras en dados ...
2,Receta de Ensalada de gulas y gambas,La ensalada de gulas y lechuga es perfecta com...,4,15,Plato principal,baja,"[1 chorro de aceite de oliva, 2 guindillas sec...",[Pon en una sartén al fuego un chorrito de ace...
3,Receta de Ensalada rusa argentina,Estamos ante una guarnición típica de las mesa...,6,45,Acompañamiento,baja,"[2 papas, 1 zanahoria, ½ taza de arvejas, ½ ta...","[Lava y pela las papas, corta en cubos de apro..."
4,Receta de Ensalada de fideos navideña,La ensalada de fideos de colores en Perú es es...,8,30,NaN,baja,"[500 gramos de fideos tornillo, 4 zanahorias, ...",[Empieza pelando y cortando las zanahorias en ...


In [52]:
def obtener_minutos(tiempo):
    # no hay nada
    if not tiempo:
        return 0
   
    # hay h y hay m
    if 'h' in tiempo and 'm' in tiempo:
        horas = int(tiempo.split(' ')[0].replace('h',''))
        minutos = int(tiempo.split(' ')[1].replace('m',''))
        return horas * 60 + minutos
        
    # hay solo h
    if 'h' in tiempo:
        horas = int(tiempo.replace('h',''))
        return horas * 60
        
    # hay solo m
    if 'm' in tiempo:
        return int(tiempo.replace('m',''))

In [57]:
for url in recetas_ok_urls:
    response = requests.get(url)
    print(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    titulos.append(soup.find("h1", class_="titulo titulo--articulo").text)
    descripciones.append(soup.find('div', class_='intro').find('p').text)
    comensales.append(int(soup.find("span", class_="property comensales").text.split(' ')[0]))
    tiempos.append(obtener_minutos(soup.find("span", class_="property duracion").text))
    try:
        categorias.append(soup.find("span", class_="property para").text)
    except:
        categorias.append(np.nan)
    dificultades.append(soup.find("span", class_="property dificultad").text.lower().replace('dificultad ', ''))
    ingredientes.append([ingrediente.text.strip() for ingrediente in soup.find('div', class_='ingredientes').find_all('label')])
    pasos.append([paso.text for paso in soup.select('.apartado .orden + p')])
    
    
df = pd.DataFrame()
df['titulo'] = titulos
df['descripcion'] = descripciones
df['comensales'] = comensales
df['tiempo_minutos'] = tiempos
df['categoria'] = categorias
df['dificultad'] = dificultades
df['ingredientes'] = ingredientes
df['pasos'] = pasos
df.head()

https://www.recetasgratis.net/receta-de-ensalada-rusa-chilena-77853.html
https://www.recetasgratis.net/receta-de-soltero-de-habas-77835.html
https://www.recetasgratis.net/receta-de-ensalada-de-gulas-y-gambas-77824.html
https://www.recetasgratis.net/receta-de-ensalada-rusa-argentina-77762.html
https://www.recetasgratis.net/receta-de-ensalada-de-fideos-navidena-77781.html
https://www.recetasgratis.net/receta-de-ensalada-blanca-navidena-77751.html
https://www.recetasgratis.net/receta-de-ensalada-de-betarraga-77742.html
https://www.recetasgratis.net/receta-de-salpicon-chileno-77653.html
https://www.recetasgratis.net/receta-de-ensalada-de-cochayuyo-77680.html
https://www.recetasgratis.net/receta-de-ensalada-chilena-77621.html
https://www.recetasgratis.net/receta-de-ensalada-coleslaw-77205.html
https://www.recetasgratis.net/receta-de-ensalada-de-quinoa-y-verduras-77158.html
https://www.recetasgratis.net/receta-de-ensalada-navidena-peruana-76980.html
https://www.recetasgratis.net/receta-de-en

,titulo,descripcion,comensales,tiempo_minutos,categoria,dificultad,ingredientes,pasos
0,Receta de Ensalada rusa chilena,La ensalada rusa chilena es un acompañamiento ...,5,30,Acompañamiento,muy baja,"[5 papas, 2 zanahorias, ½ taza de arvejas coci...","[Lava y pela las zanahorias y papas., Corta la..."
1,Receta de Soltero de habas,"El soltero de habas es un plato peruano, espec...",5,30,NaN,baja,"[½ cebolla, 1 tomate, 200 gramos de queso fres...",[Comienza picando todas las verduras en dados ...
2,Receta de Ensalada de gulas y gambas,La ensalada de gulas y lechuga es perfecta com...,4,15,Plato principal,baja,"[1 chorro de aceite de oliva, 2 guindillas sec...",[Pon en una sartén al fuego un chorrito de ace...
3,Receta de Ensalada rusa argentina,Estamos ante una guarnición típica de las mesa...,6,45,Acompañamiento,baja,"[2 papas, 1 zanahoria, ½ taza de arvejas, ½ ta...","[Lava y pela las papas, corta en cubos de apro..."
4,Receta de Ensalada de fideos navideña,La ensalada de fideos de colores en Perú es es...,8,30,NaN,baja,"[500 gramos de fideos tornillo, 4 zanahorias, ...",[Empieza pelando y cortando las zanahorias en ...


- Repetir el ejercicio para todas las recetas de todas las páginas.